In [ ]:
import os
import dotenv
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

def load_environment(env_file = ".env"):

    try:
        env_file = os.path.abspath(env_file)
        if not os.path.exists(env_file):
            print(f"Error: {env_file} does not exist")
            return False
        
        required_env_vars = ["OPENAI_API_KEY", "OPENAI_BASE_URL", "OPENAI_MODEL"]
        dotenv.load_dotenv(dotenv_path=env_file)
        
        for var in required_env_vars:
            value = os.getenv(var)
            print(f"{var}: {'is set' if value else 'is not set'}")
            if not value:
                print(f"Error: {var} is not set")
                return False
        
        return True

    except Exception as e:
        print(f"Error loading environment: {e}")
        return False

def initialize_llm():
    return OpenAI(
        model = os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        api_key = os.getenv("OPENAI_API_KEY"),
        api_base = "https://api.openai.com/v1"
    )

def main(query = "What is the capital of Italy?"):
    if not load_environment():
        print(f"Environment variables not loaded. Exiting.")
        return
    
    try:
        print("Initializing LLM...")
        llm = initialize_llm()
        print("LLM initialized successfully")
        print(f"Making API request with query: {query}")
        response = llm.complete(query)
        print(f"API response: {response}")
        openai_embed_model = OpenAIEmbedding(
            model = "text-embedding-3-small",
            api_key = os.getenv("OPENAI_API_KEY"),
            api_base = "https://api.openai.com/v1"
        )
        query_embedding = openai_embed_model.get_text_embedding(query)
        print(f"Query embedding: {query_embedding}")
        print(f"Setting global embed model...")
        Settings.embed_model = openai_embed_model
        documents = SimpleDirectoryReader("documents").load_data()
        index = VectorStoreIndex.from_documents(documents, index_batch_size=128)
        index.storage_context.persist(persist_dir = "local_vector_store")
        
        retriever = index.as_retriever()
        retrieved_docs = retriever.retrieve(query)
        print(f"Retrieved documents: {retrieved_docs}")
        print(f"Setting query engine...")
        query_engine = index.as_query_engine(
            llm = llm
        )
        print(f"Query engine set successfully")
        updated_query = "What is cool about {}".format(query)
        response = query_engine.query(updated_query)
        print(f"Query engine response: {response}")

    except Exception as e:
        print(f"Error making API request: {e}")
        print(f"Error type: {type(e)}")
    
    
if __name__ == "__main__":
    main()